In [1]:
from dotenv import load_dotenv
import os
import spotipy
import pandas as pd
import json
import time
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="b07b4796e04c4d03985727e46b29dfd2",
                                                           client_secret="035a968bce424c719ec6b686fec8f178"))


In [3]:
with open('StreamingHistoryGleb.json', encoding='utf-8') as json_file:
    data = json.load(json_file)
data = pd.DataFrame(data)

In [4]:
def get_track_info(track_name, artist_name):
    query = f"{track_name} {artist_name}"
    
    # Search for the track
    results = sp.search(q=query, type='track')
    
    # Check if any tracks were found
    if not results['tracks']['items']:
        print("No tracks found.")
        return None
    
    # Get the track ID
    track_id = results['tracks']['items'][0]['id']
    
    # Get track audio features
    track_details = sp.audio_features([track_id])[0]
    
    # Remove unnecessary fields
    track_details.pop('uri', None)
    track_details.pop('track_href', None)
    track_details.pop('analysis_url', None)
    track_details.pop('time_signature', None)
    
    track_details_df = pd.DataFrame([track_details])
    return track_details_df

In [6]:
audio_features_df = pd.DataFrame()
pieces = [data[i:i+100] for i in range(0, len(data), 100)]

for piece in pieces[10:]:
    track_info_df = pd.DataFrame()

    # Iterate over rows in the chunk
    for index, row in piece.iterrows():
        track_info = get_track_info(row['trackName'], row['artistName'])
        track_info_df = pd.concat([track_info_df,track_info],ignore_index=True)

    combined = pd.concat([piece.reset_index(drop=True), track_info_df], axis=1)

    audio_features_df = pd.concat([audio_features_df, combined], ignore_index=True)
    audio_features_df.to_csv('audio_features_combined_gleb.csv', index=False, mode='a', header=not audio_features_df.index.any(), encoding='utf-8')

    # Reset audio_features_df for the next iteration
    audio_features_df = pd.DataFrame()

    time.sleep(20)

In [7]:
def get_genre(track_id):
    track_data = sp.track(track_id)
    artist_ids = []
    for artist in track_data["artists"]:
        artist_ids.append(artist["id"])
    artists_data = sp.artists(artist_ids)
    genres = []
    for artist in artists_data["artists"]:
        genres += artist["genres"]
    return genres

In [8]:
column_names = ["endTime","artistName","trackName","msPlayed",'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'duration_ms']
df_audio = pd.read_csv("audio_features_combined_gleb.csv",names=column_names, header=None)

genres = pd.DataFrame(columns=['id', 'genre'])

# Podział ramki danych na kawałki (pieces)
pieces = [df_audio.iloc[i:i+100, :] for i in range(0, len(df_audio), 100)]
# Iteracja po kawałkach
for piece in pieces[10:]:
     #Utwórz nową kolumnę 'genre' i przypisz wyniki funkcji get_genre
    piece['genre'] = piece['id'].apply(get_genre)
    piece.to_csv('genres_last_pls_gleb.csv', index=False, mode='a', header=not genres.index.any(), encoding='utf-8')
    time.sleep(5)

C:\Users\marty\AppData\Local\Temp\ipykernel_19056\1530546653.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  piece['genre'] = piece['id'].apply(get_genre)
C:\Users\marty\AppData\Local\Temp\ipykernel_19056\1530546653.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  piece['genre'] = piece['id'].apply(get_genre)
C:\Users\marty\AppData\Local\Temp\ipykernel_19056\1530546653.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i